# Layer: Base

## @> macro from Lazy.jl
这段代码定义了一个名为 `@>` 的宏，这是一个线程宏，类似于 Julia 中的 `|>` 操作符，但更具灵活性。它允许你以一种流畅的方式将函数调用链接在一起，而不需要嵌套它们。例如，`@> x f` 等同于 `f(x)`，`@> x g f` 等同于 `f(g(x))`，以此类推。

这个宏的定义包含了几个函数：

- `thread(x)`：如果 `x` 是一个块表达式，那么它会移除所有的行并对结果进行线程处理；否则，它只是返回 `x`。
- `thread(x, ex)`：这个函数根据 `ex` 的类型来决定如何处理 `x`。如果 `ex` 是一个宏调用，那么它会将 `x` 插入到 `ex` 的参数中；如果 `ex` 是一个函数调用，那么它会将 `x` 作为第一个参数插入到 `ex` 的参数中；如果 `ex` 是一个广播调用，那么它会将 `x` 插入到 `ex` 的参数中；如果 `ex` 是一个块表达式，那么它会移除所有的行并对结果进行线程处理；否则，它会创建一个新的函数调用，其中 `ex` 是函数，`x` 是参数。
- `thread(x, exs...)`：这个函数使用 `reduce` 函数将 `thread` 函数应用到 `exs` 的所有元素上，其中 `x` 是初始值。

最后，这个宏使用 `esc` 函数来转义结果，这样在宏展开后，结果不会被误解为在宏定义的上下文中，而是在宏调用的上下文中。

需要注意的是，这个宏在 Julia 0.7 以下的版本中不可用，如果在这些版本中尝试使用它，会抛出一个错误。

In [7]:
import MacroTools: @capture, rmlines, isexpr
"""
The threading macro is like a more flexible version of the [`|>`](@ref) operator.

    @> x f = f(x)
    @> x g f == f(g(x))
    @> x a b c d e == e(d(c(b(a(x)))))

Unlike [`|>`](@ref), functions can have arguments - the value
preceding a function will be treated as its first argument

    @> x g(y, z) f == f(g(x, y, z))

    @> x g f(y, z) == f(g(x), y, z)

See Lazy.jl.

"""
macro >(exs...)
    thread(x) = isexpr(x, :block) ? thread(rmlines(x).args...) : x
  
    @static if VERSION < v"0.7"
  
      @error "The threading macro `@>` is not supported on Julia versions < 0.7"
  
    else
  
      thread(x, ex) =
      isexpr(ex, :macrocall)        ? Expr(ex.head, ex.args[1], ex.args[2], x, ex.args[3:end]...) :
      isexpr(ex, :call,)            ? Expr(ex.head, ex.args[1], x, ex.args[2:end]...) :
      @capture(ex, f_.(xs__))       ? :($f.($x, $(xs...))) :
      isexpr(ex, :block)            ? thread(x, rmlines(ex).args...) :
      Expr(:call, ex, x)
  
    end
  
    thread(x, exs...) = reduce(thread, exs, init=x)
  
    esc(thread(exs...))
end
  

@>

In [ ]:
module WorkflowProcessors
    function +(f::Function, g::Function)
        return x -> f(x) + g(x)
    end
end

# Layer: Logic


# Layer: Application


# Layer: Test

In [8]:
using Test
@testset "my @> marco test" begin
    @test 2 == @> 1 +(1)
end

Test Summary:    | Pass  Total  Time
my @> marco test |    1      1  0.0s


Test.DefaultTestSet("my @> marco test", Any[], 1, false, false, true, 1.7041688080536e9, 1.704168808053664e9)

In [26]:
@testset "param input with .|>" begin
    # broadcast an element to multiple functions
    @test (true, false) == 1 .|> (x->true, x->false)
    # exam true/false functions for all/any
    @test true == 1 .|> (x->true, x->true) |> all
    @test false == 1 .|> (x->true, x->false) |> all
    @test true == 1 .|> (x->true, x->false) |> any
    # short-circuiting cannot work with functions, because they are not lazy
    f1(x) = (println("f1 is called, and short-circuiting occurs because it return false"); false)
    f2(x) = (println("f2 is unexpectedly called, since it's not lazy"); false)
    @test false == 1 .|> (f1, f2) |> any
    # test for multiple arguments with .|>
    f1(x) =  false
    f2(x) =  false
    f1(x,y) = true
    f2(x,y) = true
    @test true == ((1,2),) .|> (f1, f2) |> all
end

f1 is called, and short-circuiting occurs because it return false
f2 is unexpectedly called, since it's not lazy
f1(x) is called
f2(x) is called
param input with .|>: Test Failed at /Users/outk/Documents/Codes/Julia/CheatSheets.jl/snippets/test.ipynb:17
  Expression: true == (((1, 2),) .|> (f1, f2)) |> all
   Evaluated: true == false
Stacktrace:
 [1] macro expansion
   @ /Applications/Julia-1.8.app/Contents/Resources/julia/share/julia/stdlib/v1.8/Test/src/Test.jl:464 [inlined]
 [2] macro expansion
   @ ~/Documents/Codes/Julia/CheatSheets.jl/snippets/test.ipynb:17 [inlined]
 [3] macro expansion
   @ /Applications/Julia-1.8.app/Contents/Resources/julia/share/julia/stdlib/v1.8/Test/src/Test.jl:1357 [inlined]
 [4] top-level scope
   @ ~/Documents/Codes/Julia/CheatSheets.jl/snippets/test.ipynb:3
Test Summary:        | Pass  Fail  Total  Time
param input with .|> |    5     1      6  0.1s


TestSetException: Some tests did not pass: 5 passed, 1 failed, 0 errored, 0 broken.

In [28]:
eltype([1])

Int64